In [ ]:
!pip install qdrant-client cohere

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import cohere

In [ ]:
from google.colab import userdata
QDRANT_API_KEY = userdata.get('QDRANT_API_KEY')
URL = userdata.get('QDRANT_URL')
COHERE_API_KEY = userdata.get('COHERE_API_KEY')

In [ ]:
client = QdrantClient(
    url=URL,
    api_key=QDRANT_API_KEY,
)

print(client.get_collections())

In [ ]:
co = cohere.Client(COHERE_API_KEY)

# Ingestion
Creating a Collection, Converting Documents to Embeddings, and Upserting the Data.

In [ ]:
client.create_collection(
    collection_name="basic-search-rerank",
    vectors_config=VectorParams(size=1024, distance=Distance.DOT),
)

# Documents to Embeddings

In [ ]:
query = "What is the purpose of feature scaling in machine learning?"

documents = [
    "In machine learning, feature scaling is the process of normalizing the range of independent variables or features. The goal is to ensure that all features contribute equally to the model, especially in algorithms like SVM or k-nearest neighbors where distance calculations matter.",

    "Feature scaling is commonly used in data preprocessing to ensure that features are on the same scale. This is particularly important for gradient descent-based algorithms where features with larger scales could disproportionately impact the cost function.",

    "In data science, feature extraction is the process of transforming raw data into a set of engineered features that can be used in predictive models. Feature scaling is related but focuses on adjusting the values of these features.",

    "Unsupervised learning algorithms, such as clustering methods, may benefit from feature scaling as it ensures that features with larger numerical ranges don't dominate the learning process.",

    "One common data preprocessing technique in data science is feature selection. Unlike feature scaling, feature selection aims to reduce the number of input variables used in a model to avoid overfitting.",

    "Principal component analysis (PCA) is a dimensionality reduction technique used in data science to reduce the number of variables. PCA works best when data is scaled, as it relies on variance which can be skewed by features on different scales.",

    "Min-max scaling is a common feature scaling technique that usually transforms features to a fixed range [0, 1]. This method is useful when the distribution of data is not Gaussian.",

    "Standardization, or z-score normalization, is another technique that transforms features into a mean of 0 and a standard deviation of 1. This method is effective for data that follows a normal distribution.",

    "Feature scaling is critical when using algorithms that rely on distances, such as k-means clustering, as unscaled features can lead to misleading results.",

    "Scaling can improve the convergence speed of gradient descent algorithms by preventing issues with different feature scales affecting the cost function's landscape.",

    "In deep learning, feature scaling helps in stabilizing the learning process, allowing for better performance and faster convergence during training.",

    "Robust scaling is another method that uses the median and the interquartile range to scale features, making it less sensitive to outliers.",

    "When working with time series data, feature scaling can help in standardizing the input data, improving model performance across different periods.",

    "Normalization is often used in image processing to scale pixel values to a range that enhances model performance in computer vision tasks.",

    "Feature scaling is significant when features have different units of measurement, such as height in centimeters and weight in kilograms.",

    "In recommendation systems, scaling features such as user ratings can improve the model's ability to find similar users or items.",

    "Dimensionality reduction techniques, like t-SNE and UMAP, often require feature scaling to visualize high-dimensional data in lower dimensions effectively.",

    "Outlier detection techniques can also benefit from feature scaling, as they can be influenced by unscaled features that have extreme values.",

    "Data preprocessing steps, including feature scaling, can significantly impact the performance of machine learning models, making it a crucial part of the modeling pipeline.",

    "In ensemble methods, like random forests, feature scaling is not strictly necessary, but it can still enhance interpretability and comparison of feature importance.",

    "Feature scaling should be applied consistently across training and test datasets to avoid data leakage and ensure reliable model evaluation.",

    "In natural language processing (NLP), scaling can be useful when working with numerical features derived from text data, such as word counts or term frequencies.",

    "Log transformation is a technique that can be applied to skewed data to stabilize variance and make the data more suitable for scaling.",

    "Data augmentation techniques in machine learning may also include scaling to ensure consistency across training datasets, especially in computer vision tasks."
]

Generate embeddings for these documents using Cohere’s embed-english-v3.0 model

In [ ]:
model = "embed-english-v3.0"

doc_embeddings = co.embed(
    texts=documents,
    model=model,
    input_type="search_document",
    embedding_types=['float']
)

# Upsert Data

In [ ]:
points = []
for idx, (embedding, doc) in enumerate(zip(doc_embeddings.embeddings.float_, documents)):
    point = PointStruct(
        id=idx,
        vector=embedding,
        payload={"document": doc}
    )
    points.append(point)

In [ ]:
operation_info = client.upsert(
    collection_name="basic-search-rerank",
    points=points
)

# Retrieval

In [ ]:
query_embeddings = co.embed(texts=[query],
                          model=model,
                          input_type="search_query",
                          embedding_types=['float'])

# Vector Search

In [ ]:
for i, point in enumerate(search_result):
    print(f"Result {i+1}:")
    print(f"  ID: {point.id}")
    print(f"  Score: {point.score:.4f}")
    print(f"  Document: {point.payload.get('document', '')[:200]}...")
    print()

# Rerank

In [ ]:
document_list = [point.payload['document'] for point in search_result]

rerank_results = co.rerank(
    model="rerank-english-v3.0",
    query=query,
    documents=document_list,
    top_n=5,
)

for i, result in enumerate(rerank_results.results):
    print(f"Rank {i+1}:")
    print(f"  Score: {result.relevance_score:.4f}")
    print(f"  Document: {document_list[result.index][:200]}...")
    print()